In [1]:
import numpy as np
import sys
import time
import h5py as h5
import matplotlib.pyplot as plt

sys.path.append(r"C:\Users\haoyuan\Documents\GitHub\CrystalDiff")

from CrystalDiff import util, pulse, lclsutil, crystal, lightpath

# Get the Crystal Geometries

In [2]:
# ----------------------------------------------------------------------------------------------------------
#                       Step 1: Pulse
# ----------------------------------------------------------------------------------------------------------
energy_center = 10.
pre_length = 1e6

# Set up the pulse
FWHM = 50 # (um)

my_pulse = pulse.GaussianPulse3D()
my_pulse.set_pulse_properties(central_energy=energy_center,
                              polar=[0., 1., 0.],
                              sigma_x=FWHM /2. / np.sqrt(np.log(2)) / util.c,
                              sigma_y=FWHM /2. / np.sqrt(np.log(2)) / util.c,
                              sigma_z=9.,
                              x0=np.array([0., 0., -pre_length]))

# ----------------------------------------------------------------------------------------------------------
#                       Step 2: Split
# ---------------------------------------------------------------------------------------------------------- 
grating_list = [crystal.RectangleGrating(), crystal.RectangleGrating()]
grating_list[0].set_a(0.5)
grating_list[0].set_b(0.5)
grating_list[1].set_a(0.5)
grating_list[1].set_b(0.5)

dtheta = np.arctan(grating_list[0].base_wave_vector[1] / my_pulse.klen0)  # This is the deviation angle.


# ----------------------------------------------------------------------------------------------------------
#                       Step 3: Delay Lines
# ----------------------------------------------------------------------------------------------------------
# Some meta data for the delay line.
h_length = 2. * np.pi / (1.9201 * 1e-4)

# Some crystal properties
chi0 = complex(-0.97631E-05, 0.14871E-06)
chih_sigma = complex(0.59310E-05, -0.14320E-06)
chihbar_sigma = complex(0.59310E-05, -0.14320E-06)
chih_pi = complex(0.46945E-05, -0.11201E-06)
chihbar_pi = complex(0.46945E-05, -0.11201E-06)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#                       Crystal for branch  1
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Set up the angles
angle_offset_1 = dtheta

# Bragg angle
bragg = np.radians(18.836) + 10e-6

d1 = 0.
d2 = 15.5e-6
d3 = 15.5e-6
d4 = 0.

# Get crystal angles
eta_b1 = np.zeros(8, dtype=np.float64)
theta_b1 = np.array([3 * np.pi / 2. - bragg,
                     np.pi / 2. - bragg,
                     np.pi / 2. + bragg + d2,
                     3 * np.pi / 2. + bragg + d2,
                     np.pi / 2. + bragg + d3,
                     3 * np.pi / 2. + bragg + d3,
                     3 * np.pi / 2. - bragg + d4,
                     np.pi / 2. - bragg + d4])

rho_b1 = theta_b1 + np.pi

asy_angle = np.deg2rad(5)
rho_b1[1] += asy_angle
rho_b1[2] -= asy_angle
rho_b1[5] -= asy_angle
rho_b1[6] += asy_angle

tau_b1 = np.zeros(8)


# Add the effect of the global rotation due to the grating momentum transfer
theta_b1 += angle_offset_1
rho_b1 += angle_offset_1

# Initialize the crystals
crystal_list_1 = lclsutil.get_crystal_list(num=8,
                                           hlen=np.array([h_length, ] * 8),
                                           theta=theta_b1,
                                           eta=eta_b1,
                                           rho=rho_b1,
                                           tau=tau_b1,
                                           chi0=chi0,
                                           chih_sigma=chih_sigma,
                                           chihbar_sigma=chihbar_sigma,
                                           chih_pi=chih_pi,
                                           chihbar_pi=chihbar_pi)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#                       Crystal for branch  2
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Set up the angles
angle_offset_2 = - dtheta
bragg = np.radians(18.836) + 13e-6

# Get crystal angles
eta_b2 = np.zeros(4, dtype=np.float64)
theta_b2 = np.array([3 * np.pi / 2. - bragg,
                     np.pi / 2. - bragg,
                     np.pi / 2. + bragg,
                     3 * np.pi / 2. + bragg])
rho_b2 = theta_b2 + np.pi
tau_b2 = np.zeros(4, dtype=np.float64)

# Add the effect of the global rotation due to the grating momentum transfer
theta_b2 += angle_offset_2
rho_b2 += angle_offset_2

# Initialize the crystals
crystal_list_2 = lclsutil.get_crystal_list(num=4,
                                           hlen=np.array([h_length, ] * 4),
                                           theta=theta_b2,
                                           eta=eta_b2,
                                           rho=rho_b2,
                                           tau=tau_b2,
                                           chi0=chi0,
                                           chih_sigma=chih_sigma,
                                           chihbar_sigma=chihbar_sigma,
                                           chih_pi=chih_pi,
                                           chihbar_pi=chihbar_pi)

# ------------------------------------------------------
#   Define the positions
# ------------------------------------------------------
path_list_fix = [5e6 - 1e5, 1.1e5, 6.5e5, 1.12e5, 6e6, 1e6]
path_list_var = [5e6, 1e4, 5e4, 1e4, 20e4, 10e4, 10e4, 10.2e4, 6e6, 1e6]
delay_time = 20

# Arrange the crystals

In [3]:
# ---------------------------------------------------------------------------
#                  Adjust the position of the crystals
# ---------------------------------------------------------------------------

(fix_branch_path,
 kout_fixed,
 intersect_fixed,
 var_branch_path,
 kout_var,
 intersect_var) = lightpath.adjust_path_length(delay_time=delay_time,
                                               fix_branch_path=path_list_fix,
                                               fix_branch_crystal=crystal_list_2,
                                               var_branch_path=path_list_var,
                                               var_branch_crystal=crystal_list_1,
                                               grating_pair=grating_list,
                                               kin=my_pulse.k0)

# -----------------------------------------------------------------------------
#                 Update the crystals
# -----------------------------------------------------------------------------
crystal_list_1 = lclsutil.update_crystal_list(crystal_list_1,
                                              surface_points=np.copy(intersect_var[1:-2]))
crystal_list_2 = lclsutil.update_crystal_list(crystal_list_2,
                                              surface_points=np.copy(intersect_fixed[1:-2]))
grating_list[0].set_surface_point(np.copy(intersect_fixed[0]))
grating_list[1].set_surface_point(np.copy(intersect_fixed[-2]))

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Get the observation point
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
observation = np.copy(intersect_fixed[-1]) 
total_path = pre_length + np.sum(fix_branch_path)

print("The total propagation length is {:.2f}m.".format(total_path / 1e6))

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#                  Change frame
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(my_pulse,
 observation,
 [crystal_list_1, 
  crystal_list_2,
  grating_list]
) = lclsutil.get_output_frame(displacement=-np.copy(intersect_fixed[-1]),
                              observe=observation,
                              pulse=my_pulse,
                              crystal_lists = [crystal_list_1, 
                                               crystal_list_2,
                                               grating_list])

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#                  Get the momentum mesh
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
number_x = 2
number_y = 800
number_z = 10 ** 5
kx_grid, ky_grid, kz_grid, axis_info = util.get_k_mesh_3d(number_x=number_x,
                                                          number_y=number_y,
                                                          number_z=number_z,
                                                          delta_e_x=1e-50,
                                                          delta_e_y=5e-4,
                                                          delta_e_z=2e-3)
kz_grid += my_pulse.klen0

# Apply fft shift
# kx_grid = np.ascontiguousarray(np.fft.fftshift(kx_grid))
kx_grid = np.zeros(1, np.float64)
number_x = 1
ky_grid = np.ascontiguousarray(np.fft.fftshift(ky_grid))
kz_grid = np.ascontiguousarray(np.fft.fftshift(kz_grid))

The total propagation length is 11.90m.


In [4]:
delta_g = 9e6 * np.sin(dtheta) / np.cos(bragg)  / 2

# Get the value for 9keV

In [5]:
delta_9 = np.arctan(grating_list[0].base_wave_vector[1] / util.kev_to_wave_number(9))
bragg_9 = np.deg2rad(21.02)

h_9 = 2 * delta_g * np.cos(bragg_9) / np.cos(delta_9)  - 9. * np.tan(delta_9) * 1e6

print("h is {:.4f} um".format(h_9))

h is -139.2960 um


# Get the value for 12 keV

In [6]:
delta_12 = np.arctan(grating_list[0].base_wave_vector[1] / util.kev_to_wave_number(12))
bragg_12 = np.deg2rad(15.6)

h_12 = 2 * delta_g * np.cos(bragg_12) / np.cos(delta_12)  - 9. * np.tan(delta_12) * 1e6

print("h is {:.4f} um".format(h_12))

h is 205.6894 um


# Get the value for 11keV

In [7]:
delta_11 = np.arctan(grating_list[0].base_wave_vector[1] / util.kev_to_wave_number(11))
bragg_11 = np.deg2rad(17)

h_11 = 2 * delta_g * np.cos(bragg_11) / np.cos(delta_11)  - 9. * np.tan(delta_11) * 1e6

print("h is {:.4f} um".format(h_11))

h is 113.0694 um


# Get the value for 10 kev

In [8]:
delta_10 = np.arctan(grating_list[0].base_wave_vector[1] / util.kev_to_wave_number(10))
bragg_10 = bragg

h_10 = 2 * delta_g * np.cos(bragg_10) / np.cos(delta_10)  - 9. * np.tan(delta_10) * 1e6

print("h is {:.4e} um".format(h_10))

h is 0.0000e+00 um


# Calculate the path length

In [9]:
energy = 10
bragg_new = bragg

g1 = 2.4 * 1e4
g2 = 2.4 * 1e4 + delta_g

d = 9e6
delta = np.arctan(grating_list[0].base_wave_vector[1] / util.kev_to_wave_number(energy))

path_length_10 = ( d / np.cos(delta)
               - 2 * (g2 - g1) * np.cos(bragg_new) * np.tan(delta) + 
               (g1 + g2) / np.sin(bragg_new) * (1- np.cos(2*bragg_new)) )

print("The total path length is {}".format(path_length_10))
print("The total propogation time is {} fs".format(path_length_10 / util.c))

The total path length is 9031376.380856752
The total propogation time is 30125428.908744432 fs


In [25]:
energy = 9

g1 = 2.4 * 1e4
g2 = 2.4 * 1e4 + delta_g

d = 9e6

path_length_9 = ( d / np.cos(delta_9)
               - 2 * (g2 - g1) * np.cos(bragg_9) * np.tan(delta_9) + 
               (g1 + g2) / np.sin(bragg_9) * (1- np.cos(2*bragg_9)) )

print("The change in path length is {} um".format(path_length_9-path_length_10))
print("The chagne in the propogation time is {} fs".format((path_length_9 - path_length_10) / util.c / 1000))

The change in path length is 3481.059402687475 um
The chagne in the propogation time is 11.611564299884671 fs


In [26]:
energy = 11

g1 = 2.4 * 1e4
g2 = 2.4 * 1e4 + delta_g

d = 9e6

path_length_11 = ( d / np.cos(delta_11)
               - 2 * (g2 - g1) * np.cos(bragg_11) * np.tan(delta_11) + 
               (g1 + g2) / np.sin(bragg_11) * (1- np.cos(2*bragg_11)) )

print("The change in path length is {} um".format(path_length_11-path_length_10))
print("The chagne in the propogation time is {} fs".format((path_length_11 - path_length_10) / util.c / 1000))

The change in path length is -2964.060200396925 um
The chagne in the propogation time is -9.887040588582535 fs


In [27]:
energy = 12

g1 = 2.4 * 1e4
g2 = 2.4 * 1e4 + delta_g

d = 9e6

path_length_12 = ( d / np.cos(delta_12)
               - 2 * (g2 - g1) * np.cos(bragg_12) * np.tan(delta_12) + 
               (g1 + g2) / np.sin(bragg_12) * (1- np.cos(2*bragg_12)) )

print("The change in path length is {} um".format(path_length_12-path_length_10))
print("The chagne in the propogation time is {} ps".format((path_length_12 - path_length_10) / util.c / 1000))

The change in path length is -5243.090276673436 um
The chagne in the propogation time is -17.489066641808034 ps


# Calculate the corresponding change in the variable branch

In [13]:
f1 = 1.2 * 1e4
f2 = 1.2 * 1e4
f3 = 1.2 * 1e4
f4 = f3 - (g2 - g1) + d * np.sin(delta_10) / np.cos(bragg_10)

# For 9 keV

In [19]:
delta_tmp = delta_9
h_tmp = h_9
bragg_tmp = bragg_9

path_9 = ( d * np.cos(delta_tmp) + h_tmp * np.sin(delta_tmp) + 
          (f1 + f2 + f3 + f4) * (1 - np.cos(2 * bragg_tmp)) / np.sin(bragg_tmp))

In [20]:
delta_tmp = delta_10
h_tmp = h_10
bragg_tmp = bragg_10

path_10 = ( d * np.cos(delta_tmp) + h_tmp * np.sin(delta_tmp) + 
          (f1 + f2 + f3 + f4) * (1 - np.cos(2 * bragg_tmp)) / np.sin(bragg_tmp))

In [21]:
delta_tmp = delta_11
h_tmp = h_11
bragg_tmp = bragg_11

path_11 = ( d * np.cos(delta_tmp) + h_tmp * np.sin(delta_tmp) + 
          (f1 + f2 + f3 + f4) * (1 - np.cos(2 * bragg_tmp)) / np.sin(bragg_tmp))

In [22]:
delta_tmp = delta_12
h_tmp = h_12
bragg_tmp = bragg_12

path_12 = ( d * np.cos(delta_tmp) + h_tmp * np.sin(delta_tmp) + 
          (f1 + f2 + f3 + f4) * (1 - np.cos(2 * bragg_tmp)) / np.sin(bragg_tmp))

# Compare with the other branch

In [28]:
print(path_9 - path_length_10)

print( (path_9 - path_length_10) / util.c / 1000 )

print( (path_length_9 - path_length_10) / util.c / 1000 )

3481.021023783833
11.61143628164199
11.611564299884671


In [29]:
print(path_10 - path_length_10)

print( (path_10 - path_length_10) / util.c / 1000 )

print( (path_length_10 - path_length_10) / util.c / 1000 )

1.862645149230957e-09
6.21311543878451e-12
0.0


In [30]:
print(path_11 - path_length_10)

print( (path_11 - path_length_10) / util.c / 1000 )

print( (path_length_11 - path_length_10) / util.c / 1000 )

-2964.0347116440535
-9.886955567254642
-9.887040588582535


In [31]:
print(path_12 - path_length_10)

print( (path_12 - path_length_10) / util.c / 1000 )

print( (path_length_12 - path_length_10) / util.c / 1000 )

-5243.047772945836
-17.48892486463364
-17.489066641808034
